In [7]:
# Import libraries - REQUIRES pip version 9.0.3
import pandas
import pandas as pd
import os
from os.path import join
import sys
import scipy.stats
import numpy
import math
import pickle
import copy
import time
import warnings
import gc

#Simplify reading/writing files
cwd=os.path.realpath(os.path.join(os.path.dirname(os.getcwd()),"..",".."))

# Using Cobrapy 0.13.0
import cobra
import cobra.test
import cobra.flux_analysis.gapfilling
from cobra import Reaction, Metabolite
from cobra.io import write_sbml_model
from cobra.flux_analysis import  flux_variability_analysis
from cobra.flux_analysis.reaction import assess_component
from cobra.manipulation.delete import *
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.medium import find_boundary_types
from cobra.util import solver as sutil


In [8]:
import math
nlog2 = math.log(2)

#Calculate doubing time from objective value
def doubling_time(model):
    with model as m:
        ov = m.slim_optimize()
        if ov < 1e-6:
            growth = 'No growth'
        else:  
            growth = nlog2*60/ov
            growth = str(round(growth, 3)) + ' minutes'
        
    return growth

In [9]:
# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

In [10]:
#Set medias
rpmi_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_cobalt2_e_','EX_cu2_e_','EX_fe3_e_','EX_mn2_e_','EX_mobd_e_','EX_zn2_e_','EX_ca2_e_','EX_no3_e_','EX_h2o_e_','EX_k_e_','EX_cl_e_','EX_mg2_e_','EX_so4_e_','EX_na1_e_','EX_pi_e_','EX_h_e_','EX_arg_L_e_','EX_asn_L_e_','EX_asp_L_e_','EX_cys_L_e_','EX_glu_L_e_','EX_gly_e_','EX_his_L_e_','EX_ile_L_e_','EX_leu_L_e_','EX_lys_L_e_','EX_met_L_e_','EX_phe_L_e_','EX_pro_L_e_','EX_ser_L_e_','EX_thr_L_e_','EX_trp_L_e_','EX_tyr_L_e_','EX_val_L_e_','EX_pnto_R_e_','EX_chol_e_','EX_inost_e_','EX_glc_D_e_','EX_gthrd_e_','EX_co2_e_'])
mdm_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_co2_e_', 'EX_cobalt2_e_', 'EX_cu2_e_' , 'EX_h_e_', 'EX_h2o_e_','EX_mn2_e_','EX_mobd_e_','EX_tungs_e_', 'EX_zn2_e_','EX_na1_e_','EX_cl_e_','EX_k_e_','EX_so4_e_','EX_nh4_e_','EX_mg2_e_','EX_pi_e_','EX_ca2_e_','EX_fe3_e_','EX_no3_e_','EX_asp_L_e_','EX_glu_L_e_','EX_arg_L_e_','EX_gly_e_','EX_ser_L_e_','EX_leu_L_e_','EX_ile_L_e_','EX_val_L_e_','EX_tyr_L_e_','EX_cys_L_e_','EX_pro_L_e_','EX_trp_L_e_','EX_thr_L_e_','EX_phe_L_e_','EX_asn_L_e_','EX_gln_L_e_','EX_his_L_e_','EX_met_L_e_','EX_ala_L_e_','EX_lys_L_e_','EX_gthrd_e_','EX_thm_e_','EX_pnto_R_e_','EX_glc_D_e_','EX_hxan_e_','EX_ura_e_'])

def rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -20.

def complete(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=-20.

def mdm(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -20.
        elif reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        elif reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -20.

def rpmi_molarity(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20. 
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -11.1012
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.2855
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -0.1503
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.3784
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -0.1359
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.1332
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.1707
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.3812
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.1737
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.0908
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.219
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.424
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.847
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -5.3655
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -108.6993
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -0.4056
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -0.4056
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -137.9818
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -6.354
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -1.1481
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -10.416
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.0967
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.3812
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1005
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.1679
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.0245
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.219
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.1104
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.001
        if 'EX_chol_e_' in reaction.id:
            reaction.lower_bound = -0.0215
        if 'EX_inost_e_' in reaction.id:
            reaction.lower_bound = -0.1943
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.0033
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.

In [11]:
# Read in  model
import cobra
model=read_model(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/NGO_513.json')

# Read in  other models to assist with annotation
NM_model = read_model(cwd+'/Gc_GENRE_2022/Models/Nmb_iTM560.json')

#Ecoli model NM model is based on - to be used for metabolite charges
ecoli = read_model(cwd+'/Gc_GENRE_2022/Models/ecoli.json')

#Universal model for adding reactions
universalmodel= read_model(cwd+'/Gc_GENRE_2022/Models/universal_model_cobrapy.json')

#Downloaded carvememodel for Neisseria gonorrhoeae FA1090 from PMID: 30192979 to compare annotations and add Gc specific reactions
carveme_model= cobra.io.read_sbml_model(cwd+'/Gc_GENRE_2022/Models/Neisseria_gonorrhoeae_FA_1090.xml.gz')
carvememodel=carveme_model
cobra.io.save_json_model(carveme_model, cwd+'/Gc_GENRE_2022/Models/Neisseria_gonorrhoeae_FA_1090.json')

In [12]:
rpmi_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_cobalt2_e_','EX_cu2_e_','EX_fe3_e_','EX_mn2_e_','EX_mobd_e_','EX_zn2_e_','EX_ca2_e_','EX_no3_e_','EX_h2o_e_','EX_k_e_','EX_cl_e_','EX_mg2_e_','EX_so4_e_','EX_na1_e_','EX_pi_e_','EX_h_e_','EX_arg_L_e_','EX_asn_L_e_','EX_asp_L_e_','EX_cys_L_e_','EX_glu_L_e_','EX_gly_e_','EX_his_L_e_','EX_ile_L_e_','EX_leu_L_e_','EX_lys_L_e_','EX_met_L_e_','EX_phe_L_e_','EX_pro_L_e_','EX_ser_L_e_','EX_thr_L_e_','EX_trp_L_e_','EX_tyr_L_e_','EX_val_L_e_','EX_pnto_R_e_','EX_chol_e_','EX_inost_e_','EX_glc_D_e_','EX_gthrd_e_','EX_co2_e_'])

def rpmi_with_lactate_only(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -20.
            
def rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -20.


In [13]:
def complete(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=-1000.
            
complete(model)
model.optimize()

,fluxes,reduced_costs
12DGR120tipp,0.000000,0.000000
12DGR140tipp,0.000000,0.000000
12DGR141tipp,0.000000,0.000000
12DGR160tipp,0.000000,0.000000
12DGR161tipp,0.000000,0.000000
...,...,...
Zn2tex,0.176801,0.000000
EX_apoACP_c_,-2.057390,0.000000
GLCTR2,0.000000,-0.031651
GALT3,0.000000,-0.031651


In [15]:
#Identify genes/reactions to add to the model from carveme
model_genes  = []
for gene in model.genes:
    model_genes.append(gene.id)

#Get list of genes from Carveme model for FA1090 in NGO format
carveme_model_genes= list()
for gene in carveme_model.genes:
    carveme_model_genes.append(gene.id)
carveme_model_genes

#Use look_back table of annotations from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5399841/
annotation_table = pd.read_csv (cwd+'/Gc_GENRE_2022/Curate_Gc_Model/Annotation_resources/look_back_table.csv', usecols= ['Locus Tag ID','Protein.ID'])
annotation_table=annotation_table.rename(columns={'Protein.ID': 'ProteinID', 'Locus Tag ID': "NGO_ID"})
annotation_table
                        
#Match id format between kegg and model
annotation_table['ProteinID'] = annotation_table['ProteinID'].str.replace('.', '_')

annotatedgenes= annotation_table[annotation_table.ProteinID.isin(carveme_model_genes)]
converted_carveme_model_genes= list()
for tag in annotatedgenes['NGO_ID']:
        converted_carveme_model_genes.append(tag)
        
unique_genes_carveme = set(converted_carveme_model_genes) - set(model_genes) # correct elements, but not yet in sorted order
print(sorted(unique_genes_carveme)) # sort and print



['NGO0057', 'NGO0059', 'NGO0063', 'NGO0084', 'NGO0085', 'NGO0114', 'NGO0176', 'NGO0177', 'NGO0186', 'NGO0188', 'NGO0189', 'NGO0199', 'NGO0231', 'NGO0294', 'NGO0351', 'NGO0382', 'NGO0387', 'NGO0388', 'NGO0541', 'NGO0567', 'NGO0571', 'NGO0604', 'NGO0608', 'NGO0632', 'NGO0633', 'NGO0652', 'NGO0653', 'NGO0659', 'NGO0676', 'NGO0718', 'NGO0763', 'NGO0829', 'NGO0830', 'NGO0842', 'NGO0858', 'NGO0886', 'NGO0902', 'NGO0904', 'NGO0905', 'NGO0906', 'NGO0934', 'NGO0941', 'NGO0974', 'NGO1032', 'NGO1033', 'NGO1047', 'NGO1052', 'NGO1055', 'NGO1154', 'NGO1204', 'NGO1207', 'NGO1226', 'NGO1227', 'NGO1243', 'NGO1298', 'NGO1323', 'NGO1325', 'NGO1334', 'NGO1364', 'NGO1365', 'NGO1381', 'NGO1404', 'NGO1432', 'NGO1433', 'NGO1434', 'NGO1454', 'NGO1530', 'NGO1559', 'NGO1577', 'NGO1684', 'NGO1686', 'NGO1692', 'NGO1722', 'NGO1732', 'NGO1733', 'NGO1753', 'NGO1769', 'NGO1858', 'NGO1866', 'NGO1922', 'NGO1930', 'NGO1963', 'NGO1977', 'NGO1990', 'NGO2011', 'NGO2012', 'NGO2013', 'NGO2014', 'NGO2015', 'NGO2027', 'NGO2033'

<ipython-input-15-1829916bf6eb>:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  annotation_table['ProteinID'] = annotation_table['ProteinID'].str.replace('.', '_')


In [16]:
#Add NGOIDs to the carveme model as an annotation
for gene in annotatedgenes['ProteinID']:
    if gene in carveme_model.genes:
        row=annotatedgenes.index[annotatedgenes['ProteinID']==gene]
        id=annotatedgenes['NGO_ID']
        annotation =id.loc[row].item()
        carveme_model.genes.get_by_id(gene).annotation['NGO_ID']= annotation

In [17]:
carvemegenes = []
carvemengoids=[]
carvemereactions = []


for gene in carveme_model.genes:
    carvemegenes.append(gene.id)
    carvemengoids.append(gene.annotation)
    sets=list(gene.reactions)
    rxnlist=[]
    for x in sets:
        rxnlist.append(x)
    carvemereactions.append(rxnlist)

dict = {'carvemegenes': carvemegenes, 'NGO_IDs':carvemengoids,'carvemereactions':carvemereactions}
carvemegenes_reactions_DF = pandas.DataFrame(dict)
carvemegenes_reactions_DF.to_csv(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/Annotation_resources/carveme_genes_reactions.csv')
carvemegenes_reactions_DF

,carvemegenes,NGO_IDs,carvemereactions
0,YP_208072_1,{'NGO_ID': 'NGO0974'},[2AGPGAT160: 2agpg160_c + atp_c + hdca_c --> a...
1,YP_208834_1,{'NGO_ID': 'NGO1804'},"[3HAD40: 3haACP_c --> but2eACP_c + h2o_c, 3HAD..."
2,YP_209169_1,{'NGO_ID': 'NGO2163'},[3OAR121: 3ocddec5eACP_c + h_c + nadph_c --> 3...
3,YP_208796_1,{'NGO_ID': 'NGO1763'},[KAS15: accoa_c + h_c + malACP_c --> actACP_c ...
4,YP_209174_1,{'NGO_ID': 'NGO2168'},[3OAS180: h_c + malACP_c + palmACP_c --> 3ooct...
...,...,...,...
505,YP_207518_1,{'NGO_ID': 'NGO0362'},[UPPDC1: 4.0 h_c + uppg3_c --> 4.0 co2_c + cpp...
506,YP_207509_1,{'NGO_ID': 'NGO0353'},[UPPRT: prpp_c + ura_c --> ppi_c + ump_c]
507,YP_207732_1,{'NGO_ID': 'NGO0589'},"[URAt: ura_e <=> ura_c, URAt2: h_e + ura_e -->..."
508,YP_208502_1,{'NGO_ID': 'NGO1448'},[USHD: h2o_c + u23ga_c --> 2.0 h_c + lipidX_c ...


In [18]:
carveme_model.metabolites.ala__L_c.id ='ala_L_c'
carveme_model.metabolites.glu__L_c.id ='glu_L_c'

In [19]:
ALATA_L =carvememodel.reactions.get_by_id('ALATA_L')
carvememodel.reactions.get_by_id('ALATA_L').gene_reaction_rule='NGO1047'
model.add_reactions([ALATA_L])
model.reactions.get_by_id('ALATA_L')

Reaction identifier,ALATA_L
Name,L-alanine transaminase
Memory address,0x01fcbe948d90
Stoichiometry,akg_c + ala_L_c <=> glu_L_c + pyr_c 2-Oxoglutarate + L-Alanine <=> L-Glutamate + Pyruvate
GPR,NGO1047
Lower bound,-1000.0
Upper bound,1000.0


In [20]:
AP4AH =carvememodel.reactions.get_by_id('AP4AH')
carvememodel.reactions.get_by_id('AP4AH').gene_reaction_rule='NGO0231'
model.add_reactions([AP4AH])
model.reactions.get_by_id('AP4AH')


Reaction identifier,AP4AH
Name,Ap4A hydrolase
Memory address,0x01fcb2e027f0
Stoichiometry,"ap4a_c + h2o_c --> 2.0 adp_c + 2.0 h_c P1,P4-Bis(5'-adenosyl) tetraphosphate + H2O --> 2.0 ADP + 2.0 H"
GPR,NGO0231
Lower bound,0.0
Upper bound,1000.0


In [21]:
#ANS already exists in annotated Gc model, the model GPR should require NGO9872 and NGO1204
print (model.reactions.get_by_id('ANS').gene_reaction_rule)
model.reactions.get_by_id('ANS').gene_reaction_rule='NGO0872 and NGO1204'
print (model.reactions.get_by_id('ANS').gene_reaction_rule)

NGO0872
NGO0872 and NGO1204


In [22]:
BTDD_RR =carvememodel.reactions.get_by_id('BTDD_RR')
carvememodel.reactions.get_by_id('BTDD_RR').gene_reaction_rule='NGO0186'
model.add_reactions([BTDD_RR])
model.reactions.get_by_id('BTDD_RR')

Reaction identifier,BTDD_RR
Name,R R butanediol dehydrogenase
Memory address,0x01fcb4899d60
Stoichiometry,btd_RR_c + nad_c <=> actn__R_c + h_c + nadh_c R R 2 3 Butanediol C4H10O2 + Nicotinamide-adenine-dinucleotide <=> R Acetoin C4H8O2 + H + Nicotinamide-adenine-dinucleotide--reduced
GPR,NGO0186
Lower bound,-1000.0
Upper bound,1000.0


In [23]:
CYTOM =carvememodel.reactions.get_by_id('CYTOM')
carvememodel.reactions.get_by_id('CYTOM').gene_reaction_rule='NGO0676'
model.add_reactions([CYTOM])
model.reactions.get_by_id('CYTOM')

Reaction identifier,CYTOM
Name,Cytosine 5 methyltransferase
Memory address,0x01fcb4bdac70
Stoichiometry,amet_c + csn_c <=> 5mcsn_c + ahcys_c + h_c S-Adenosyl-L-methionine + Cytosine <=> 5-Methylcytosine + S-Adenosyl-L-homocysteine + H
GPR,NGO0676
Lower bound,-1000.0
Upper bound,1000.0


In [24]:
#The gene currently indicated for this reaction is NGO1506 - however KEGG annotates this gene as a 8-oxo-dGTP diphosphatase (MutT), KEGG reaction 	 R09832
#CARVEME indicates this reactions is produced by NGO1334. However this gene is annotated as a putative RNA pyrophosphohydrolase. The function isn't clear. 
# Replacing with the KEGG annotated gene for this reaction NGO0224

DNTPPA =carvememodel.reactions.get_by_id('DNTPPA')
model.reactions.get_by_id('DNTPPA').gene_reaction_rule='NGO0224'
model.reactions.get_by_id('DNTPPA')


Reaction identifier,DNTPPA
Name,R_DNTPPA
Memory address,0x01fcac579af0
Stoichiometry,ahdt_c + h2o_c --> dhpmp_c + h_c + ppi_c 2-Amino-4-hydroxy-6-erythro-1-2-3-trihydroxypropyl-dihydropteridine-triphosphate + H2O --> Dihydroneopterin-monophosphate + H + Diphosphate
GPR,NGO0224
Lower bound,0.0
Upper bound,999999.0


In [25]:
#DNMPPA is not annotated in KEGG, but is predicted to use an enzyme with E.C. number 3.6.1-, and is listed as an orphan gene in the current model. 
#I am tentatively de-orphaning the gene by adding the annotation from CARVEME for NGO1334

DNMPPA =carvememodel.reactions.get_by_id('DNMPPA')
model.reactions.get_by_id('DNMPPA').gene_reaction_rule='NGO1334'
model.reactions.get_by_id('DNMPPA')

Reaction identifier,DNMPPA
Name,R_DNMPPA
Memory address,0x01fcac579580
Stoichiometry,dhpmp_c + h2o_c --> dhnpt_c + pi_c Dihydroneopterin-monophosphate + H2O --> Dihydroneopterin + Phosphate
GPR,NGO1334
Lower bound,0.0
Upper bound,999999.0


In [26]:
#CARVEME indicates FADRx uses NGO0059 or NGO0659. However NGO0659 has different annotations in KEGG that do not match this function. Removed NGO0659. 
FADRx =carvememodel.reactions.get_by_id('FADRx')
carvememodel.reactions.get_by_id('FADRx').gene_reaction_rule='NGO0059'
model.add_reactions([FADRx])
model.reactions.get_by_id('FADRx')

Reaction identifier,FADRx
Name,FAD reductase
Memory address,0x01fcb2e634f0
Stoichiometry,fad_c + h_c + nadh_c --> fadh2_c + nad_c Flavin-adenine-dinucleotide-oxidized + H + Nicotinamide-adenine-dinucleotide--reduced --> Flavin-adenine-dinucleotide-reduced + Nicotinamide-adenine-dinucleotide
GPR,NGO0059
Lower bound,0.0
Upper bound,1000.0


In [27]:
#CARVEME and KEGG indicate presence of this reaction
HPA3MO =carvememodel.reactions.get_by_id('HPA3MO')
carvememodel.reactions.get_by_id('HPA3MO').gene_reaction_rule='NGO0059'
model.add_reactions([HPA3MO])
model.reactions.get_by_id('HPA3MO')

Reaction identifier,HPA3MO
Name,4 hydroxyphenylacetate 3 monooxygenase nadh
Memory address,0x01fcb30f5100
Stoichiometry,4hphac_c + h_c + nadh_c + o2_c --> 34dhpha_c + h2o_c + nad_c 4-Hydroxyphenylacetate + H + Nicotinamide-adenine-dinucleotide--reduced + O2 --> 3-4-Dihydroxyphenylacetate + H2O + Nicotinamide-adenine-dinucleotide
GPR,NGO0059
Lower bound,0.0
Upper bound,1000.0


In [28]:
#Model has NGO1406 annotated for GLYCL. CARVEME has NGO1325 annotated. 
#The Glycine cleavage system is a functional system composed of 4 different components *H, P, T, and L. https://www.biocyc.org/GCF_000006845/NEW-IMAGE?type=PATHWAY&object=GLYCLEAV-PWY
#All 4 components should be required for function of this system. As such, I am listing ALL 6 genes, indicated in KEGG, for this reaction. 

GLYCL =carvememodel.reactions.get_by_id('GLYCL')
model.reactions.get_by_id('GLYCL').gene_reaction_rule='NGO0562 and NGO0915 and NGO0925 and NGO1325 and NGO1404 and NGO1406'
model.reactions.get_by_id('GLYCL')

Reaction identifier,GLYCL
Name,R_GLYCL
Memory address,0x01fcac671ca0
Stoichiometry,gly_c + nad_c + thf_c <=> co2_c + mlthf_c + nadh_c + nh4_c Glycine + Nicotinamide-adenine-dinucleotide + 5-6-7-8-Tetrahydrofolate <=> CO2 + 5-10-Methylenetetrahydrofolate + Nicotinamide-adenine-dinucleotide--reduced + Ammonium
GPR,NGO0562 and NGO0915 and NGO0925 and NGO1325 and NGO1404 and NGO1406
Lower bound,-999999.0
Upper bound,999999.0


In [29]:
#CARVEME indicated NGO2033
#KEGG indicates NGO1187 AND NGO2033 participate in this reaction https://www.genome.jp/entry/ngo:NGO_1187+ngo:NGO_2033


GLYOX =carvememodel.reactions.get_by_id('GLYOX')
carvememodel.reactions.get_by_id('GLYOX')

Reaction identifier,GLYOX
Name,Hydroxyacylglutathione hydrolase
Memory address,0x01fcb3073d90
Stoichiometry,h2o_c + lgt__S_c --> gthrd_c + h_c + lac__D_c H2O + (R)-S-Lactoylglutathione --> Reduced glutathione + H+ + D-Lactate
GPR,YP_208259_1 or YP_209053_1
Lower bound,0.0
Upper bound,1000.0


In [30]:
model.reactions.get_by_id('HPA3MO').gene_reaction_rule='NGO1187 and NGO2033'
model.reactions.get_by_id('GLYOX')

Reaction identifier,GLYOX
Name,R_GLYOX
Memory address,0x01fcac67d460
Stoichiometry,h2o_c + lgt_S_c <=> gthrd_c + h_c + lac_D_c H2O + R-S-Lactoylglutathione <=> Reduced-glutathione + H + D-Lactate
GPR,NGO1187
Lower bound,-999999.0
Upper bound,999999.0


In [31]:
#CARVEME indicates NGO0387 as performing this reaction, and so does KEGG. https://www.genome.jp/entry/ngo:NGO_0387
GTPCI =carvememodel.reactions.get_by_id('GTPCI')
model.reactions.get_by_id('GTPCI').gene_reaction_rule='NGO0387'
model.reactions.get_by_id('GTPCI')

Reaction identifier,GTPCI
Name,R_GTPCI
Memory address,0x01fcac68a910
Stoichiometry,gtp_c + h2o_c --> ahdt_c + for_c + h_c GTP + H2O --> 2-Amino-4-hydroxy-6-erythro-1-2-3-trihydroxypropyl-dihydropteridine-triphosphate + Formate + H
GPR,NGO0387
Lower bound,0.0
Upper bound,999999.0


In [32]:
#Adding reactions according to PMCID: PMC4697154/PMID: 26567338 which are conducted by plsX (Ngo2151)
x = ['ACPPAT161' , 'G3PAT120' , 'ACPPAT181' , 'G3PAT180' , 'ACPPAT120' , 'G3PAT181' , 'ACPPAT141' , 'G3PAT140' , 'ACPPAT160' , 'G3PAT161' , 'G3PAT160' , 'ACPPAT140' , 'ACPPAT180']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO2171'
    model.add_reactions([reaction])

Ignoring reaction 'G3PAT120' since it already exists.
Ignoring reaction 'G3PAT180' since it already exists.
Ignoring reaction 'G3PAT181' since it already exists.
Ignoring reaction 'G3PAT140' since it already exists.
Ignoring reaction 'G3PAT160' since it already exists.


In [33]:
#providing gene for orphan reactions according to PMCID: PMC4697154/PMID: 26567338 which are conducted by plsX (Ngo2151)
x = ['G3PAT120' , 'G3PAT180' , 'G3PAT181' , 'G3PAT140' , 'G3PAT161' , 'G3PAT160' ]
for r in x:
    reaction =model.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO2171'


In [34]:
#Adding reactions according to PMCID: PMC4697154/PMID: 26567338 which are conducted by plsY (Ngo2151)
x = ['APG3PAT181' , 'APG3PAT160' , 'APG3PAT180' , 'APG3PAT120' , 'APG3PAT141' , 'APG3PAT161' , 'APG3PAT140']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0858'
    model.add_reactions([reaction])


In [35]:
#Adding peptidase reaction
x = ['AMPTASEPG']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO1227'
    model.add_reactions([reaction])

In [36]:
#cardiolipin has been in cultures of Gc PMCID: PMC235958/ PMID: 810478
x = ['CLPNS181pp' , 'CLPNS141pp' , 'CLPNS140pp' , 'CLPNS160pp' , 'CLPNS120pp' , 'CLPNS161pp' , 'CLPNS180pp']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0902'
    model.add_reactions([reaction])

In [37]:
#reduction of H2O2 to water facilitated by cytochrome c peroxidase, with Laz as an electron donor. Laz is encoded by NGO0994
reaction = cobra.Reaction('CYPOX')
reaction.name = 'Cytochrome C Peroxidase'
reaction.lower_bound = 0.  
reaction.upper_bound = 1000.  
reaction.gene_reaction_rule='NGO1769 and NGO0994'



h_c=model.metabolites.get_by_id('h_c')
h2o2= model.metabolites.get_by_id('h2o2_c')
h2o_c=model.metabolites.get_by_id('h2o_c')

reaction.add_metabolites({
    h2o2: -1.0,
    h_c: -2.0,
    h2o_c: 2.0,
})

model.add_reactions([reaction])
print(model.reactions.CYPOX)
print(model.reactions.CYPOX.check_mass_balance())

CYPOX: h2o2_c + 2.0 h_c --> 2.0 h2o_c
{}


In [38]:
x = ['FLVR', 'FLVRx']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0388'
    model.add_reactions([reaction])

In [39]:
#This reaction is indicated as a bidirectional reaction in Bigg. However Kegg indicates it is 
#unidirection with NGO1977 performing the forward reaction and NGO1406 the reverse reaction (https://www.kegg.jp/pathway/ngo00670+NGO_1977)
x = ['FOMETRi']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO1977 and NGO1406'
    model.add_reactions([reaction])

In [40]:
model.reactions.GLYCLTDx.gene_reaction_rule='NGO2043 or NGO1243'

In [41]:
x = ['GRXR']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0114 or NGO1381 or NGO0351'
    model.add_reactions([reaction])

In [42]:
#Carveme did not correctly annotate NGO1684, (its Not GTPCI) but it does perform a reaction just downstream. 
#Involved in folate biosynthesis. Corrected and added the full pathway.
#To include NGO0387 (GTPCI), NGO0131, NGO00720, NGO0129, NGO1684 (https://www.genome.jp/pathway/ngo00790+NGO_1684)
x = ['PTHPS']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0131'
    model.add_reactions([reaction])
x = ['CPH4S']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0131'
    model.add_reactions([reaction])
x = ['CDGS']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO00720'
    model.add_reactions([reaction])
x = ['CDGR']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO1684'
    model.add_reactions([reaction])
x = ['CCGS']
for r in x:
    reaction =universalmodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0129'
    model.add_reactions([reaction])

In [43]:
model.reactions.Kt2pp.gene_reaction_rule='NGO1774 or NGO1154'

In [44]:
x = ['MLDEP2pp']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0571 or NGO1686'
    model.add_reactions([reaction])
#ltgD (0626) and ltgC (2048) ltgB (1033) and ltgE(0608) are all murein lytic transcglycosylases. PMCID: PMC3412582/PMID: 22432703
#Adding all of them as potential genes 
x = ['MLTGY3pp']
for r in x:
    reaction =carvememodel.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0608 or NGO1033 or NGO0626 or NGO2048'
    model.add_reactions([reaction])

x = ['MLTGY1pp' ,'MLTGY4pp' , 'MLTGY2pp']
for r in x:
    reaction =model.reactions.get_by_id(r)
    reaction.gene_reaction_rule='NGO0608 or NGO1033 or NGO0626 or NGO2048'

In [45]:
#Deorphan PPPGO
model.reactions.PPPGO.gene_reaction_rule= 'NGO1692'

In [46]:
#Add additional thioredoxin proteins
model.reactions.THIORDXi.gene_reaction_rule=  'NGO0328 or NGO1923 or NGO0652 or NGO0057'

In [47]:
#CARVEME indicates NGO1052
#KEGG indicates NGO0073 and NGO1052
#NGO0203 has a different EC number - EC 3.1.3.105 . I am replacing with the KEGG/CARVEME gene annotations

PGLYCP =carvememodel.reactions.get_by_id('PGLYCP')

model.reactions.get_by_id('PGLYCP').gene_reaction_rule='NGO0073 or NGO1052'
model.reactions.get_by_id('PGLYCP')

Reaction identifier,PGLYCP
Name,R_PGLYCP
Memory address,0x01fcac7bffd0
Stoichiometry,2pglyc_c + h2o_c --> glyclt_c + pi_c 2-Phosphoglycolate + H2O --> Glycolate + Phosphate
GPR,NGO0073 or NGO1052
Lower bound,0.0
Upper bound,999999.0


In [48]:
#CARVEME indicates NGO0435 or NGO1752. Model has mislabeled NGO number with an underscore. Replacing.
SPMS =carvememodel.reactions.get_by_id('SPMS')
model.reactions.get_by_id('SPMS').gene_reaction_rule='NGO0435 or NGO1752'
model.reactions.get_by_id('SPMS')

Reaction identifier,SPMS
Name,R_SPMS
Memory address,0x01fcac877c40
Stoichiometry,ametam_c + ptrc_c --> 5mta_c + h_c + spmd_c S-Adenosylmethioninamine + Putrescine --> 5-Methylthioadenosine + H + Spermidine
GPR,NGO0435 or NGO1752
Lower bound,0.0
Upper bound,999999.0


In [49]:
#Model contains ethanol production - however Gc does not produce ethanol. Removing this reaction.
ALCD2x =model.reactions.get_by_id('ALCD2x')
model.remove_reactions([ALCD2x])



In [50]:
#Similarly, the mening model had added acetylaldehyde production, without genetic evidence, to support the production of ethanol observed in mening. 
#Because Gc does not produce ethanol or acetylaldehyde, removing these associated reaction.
ALDD2y =model.reactions.get_by_id('ALDD2y')
model.remove_reactions([ALDD2y])

ACALDtpp =model.reactions.get_by_id('ACALDtpp')
model.remove_reactions([ACALDtpp])

EX_acald_e_ =model.reactions.get_by_id('EX_acald_e_')
model.remove_reactions([EX_acald_e_])

In [51]:
rpmi(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
apoACP_c,EX_apoACP_c_,0.02302,0,0.00%
arg_L_e,EX_arg_L_e_,0.2092,6,0.33%
asn_L_e,EX_asn_L_e_,0.5166,4,0.54%
asp_L_e,EX_asp_L_e_,20,4,21.06%
ca2_e,EX_ca2_e_,0.002968,0,0.00%
cl_e,EX_cl_e_,0.002968,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.001978,0,0.00%
cu2_e,EX_cu2_e_,0.001978,0,0.00%
cys_L_e,EX_cys_L_e_,20,3,15.79%
fe3_e,EX_fe3_e_,0.009183,0,0.00%


In [52]:
#CARVEME revealed that NGO0904-NGO0906 is not present in the Gc model - which was recently discovered as an Gc
#lactate dehydrogenase. Additionally other aspects of lactate metabolism are corrected here. 

#PMCID: PMC4215069
#PMCID: PMC7530546

In [53]:
model.metabolites.get_by_id('lac_D_c')

Metabolite identifier,lac_D_c
Name,D-Lactate
Memory address,0x01fcabf36910
Formula,C3H5O3
Compartment,cytosol
In 3 reaction(s),"LDH_D, GLYOX, DLACt2pp"


In [54]:
carvememodel.metabolites.get_by_id('lac__D_c').id='lac_D_c'

In [55]:
model.reactions.get_by_id('L_LACD4')

Reaction identifier,L_LACD4
Name,R_L_LACD4
Memory address,0x01fcac6d4bb0
Stoichiometry,lac_L_c + nad_c --> h_c + nadh_c + pyr_c L-Lactate + Nicotinamide-adenine-dinucleotide --> H + Nicotinamide-adenine-dinucleotide--reduced + Pyruvate
GPR,NGO0639 or Orphan
Lower bound,0.0
Upper bound,999999.0


In [56]:
#NGO0639 is a quinone depdendent L-lactate dehydrogenase. There are no known NAD/NADH dependent L-lactate dehydrogenases in Gc.
LACD4 =model.reactions.get_by_id('L_LACD4')
model.remove_reactions([LACD4])
rpmi_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
apoACP_c,EX_apoACP_c_,0.01471,0,0.00%
arg_L_e,EX_arg_L_e_,0.1337,6,1.03%
asn_L_e,EX_asn_L_e_,0.1007,4,0.52%
asp_L_e,EX_asp_L_e_,0.2687,4,1.38%
ca2_e,EX_ca2_e_,0.001896,0,0.00%
cl_e,EX_cl_e_,0.001896,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.001264,0,0.00%
cu2_e,EX_cu2_e_,0.001264,0,0.00%
cys_L_e,EX_cys_L_e_,0.03932,3,0.15%
fe3_e,EX_fe3_e_,0.005867,0,0.00%


In [57]:
#The reaction LDH_D is a the NAD dependent conversion of D_lac to pyruvate. It is completed by ldhA (NGO1336). 
#This reaction should be bidirectional. 

model.reactions.get_by_id('LDH_D')
model.reactions.get_by_id('LDH_D').gene_reaction_rule='NGO1336'
model.reactions.get_by_id('LDH_D').lower_bound =-1000.
model.reactions.get_by_id('LDH_D')

Reaction identifier,LDH_D
Name,R_LDH_D
Memory address,0x01fcac6d4fd0
Stoichiometry,lac_D_c + nad_c <=> h_c + nadh_c + pyr_c D-Lactate + Nicotinamide-adenine-dinucleotide <=> H + Nicotinamide-adenine-dinucleotide--reduced + Pyruvate
GPR,NGO1336
Lower bound,-1000.0
Upper bound,999999.0


In [58]:
rpmi_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
apoACP_c,EX_apoACP_c_,0.01471,0,0.00%
arg_L_e,EX_arg_L_e_,0.1337,6,1.03%
asn_L_e,EX_asn_L_e_,0.1007,4,0.52%
asp_L_e,EX_asp_L_e_,0.2687,4,1.38%
ca2_e,EX_ca2_e_,0.001896,0,0.00%
cl_e,EX_cl_e_,0.001896,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.001264,0,0.00%
cu2_e,EX_cu2_e_,0.001264,0,0.00%
cys_L_e,EX_cys_L_e_,0.03932,3,0.15%
fe3_e,EX_fe3_e_,0.005867,0,0.00%


In [59]:
#The reaction LDH_D2 is competed by ldhD (NGO0890), a Quinone dependent D-lactate dehydrogenase.
LDH_D2=carvememodel.reactions.get_by_id('LDH_D2')
carvememodel.reactions.get_by_id('LDH_D2').gene_reaction_rule='NGO0890'
model.add_reactions([LDH_D2])
model.reactions.get_by_id('LDH_D2')


Reaction identifier,LDH_D2
Name,D-lactate dehydrogenase
Memory address,0x01fcb3583df0
Stoichiometry,lac_D_c + q8_c --> pyr_c + q8h2_c D-Lactate + Ubiquinone-8 --> Pyruvate + Ubiquinol-8
GPR,NGO0890
Lower bound,0.0
Upper bound,1000.0


In [60]:
rpmi_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
apoACP_c,EX_apoACP_c_,0.01471,0,0.00%
arg_L_e,EX_arg_L_e_,0.1337,6,1.03%
asn_L_e,EX_asn_L_e_,0.1007,4,0.52%
asp_L_e,EX_asp_L_e_,0.2687,4,1.38%
ca2_e,EX_ca2_e_,0.001896,0,0.00%
cl_e,EX_cl_e_,0.001896,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.001264,0,0.00%
cu2_e,EX_cu2_e_,0.001264,0,0.00%
cys_L_e,EX_cys_L_e_,0.03932,3,0.15%
fe3_e,EX_fe3_e_,0.005867,0,0.00%


In [61]:
#L_LACD2 is done by lldD (NGO0639), and l-lactate dehydrogenase that is quinone dependent. 
#IT was also found that lutCAB (NGO0904-NGO0906) are also L-lactate dehydrogenases, predicted to be quinone dependent.
# These are added here. 
model.reactions.get_by_id('L_LACD2').gene_reaction_rule='NGO0639 or (NGO0904 and NGO0905 and NGO0906)'
model.reactions.get_by_id('L_LACD2')



Reaction identifier,L_LACD2
Name,R_L_LACD2
Memory address,0x01fcac6d4a90
Stoichiometry,lac_L_c + q8_c --> pyr_c + q8h2_c L-Lactate + Ubiquinone-8 --> Pyruvate + Ubiquinol-8
GPR,NGO0639 or (NGO0904 and NGO0905 and NGO0906)
Lower bound,0.0
Upper bound,999999.0


In [62]:
rpmi_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
apoACP_c,EX_apoACP_c_,0.01471,0,0.00%
arg_L_e,EX_arg_L_e_,0.1337,6,1.03%
asn_L_e,EX_asn_L_e_,0.1007,4,0.52%
asp_L_e,EX_asp_L_e_,0.2687,4,1.38%
ca2_e,EX_ca2_e_,0.001896,0,0.00%
cl_e,EX_cl_e_,0.001896,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.001264,0,0.00%
cu2_e,EX_cu2_e_,0.001264,0,0.00%
cys_L_e,EX_cys_L_e_,0.03932,3,0.15%
fe3_e,EX_fe3_e_,0.005867,0,0.00%


In [63]:
model.genes.get_by_id('NGO1449')

Gene identifier,NGO1449
Name,
Memory address,0x01fcac406430
Functional,True
In 3 reaction(s),"LLACt3pp, GLYCLTt2rpp, DLACt2pp"


In [64]:
#L-LActate transport is abolished in an lctP (NGO1449) mutant, therefore, removing NGO1361 from L-lactate transport.
#NGO1361 is homologous to the L-lactate transporter lldP from E. coli, however it is much shorter in Gc, only 71 amino acids. It appears to be truncated/nonfunctioal. 
# So I am removing it also from DLACt2pp transport.

model.reactions.get_by_id('LLACt3pp')
model.reactions.get_by_id('LLACt3pp').gene_reaction_rule='NGO1449'
model.reactions.get_by_id('LLACt3pp')

model.reactions.get_by_id('DLACt2pp')
model.reactions.get_by_id('DLACt2pp').gene_reaction_rule='NGO1449'
model.reactions.get_by_id('DLACt2pp')



Reaction identifier,DLACt2pp
Name,R_DLACt2pp
Memory address,0x01fcac56cf70
Stoichiometry,h_p + lac_D_p <=> h_c + lac_D_c H + D-Lactate <=> H + D-Lactate
GPR,NGO1449
Lower bound,-999999.0
Upper bound,999999.0


In [65]:
rpmi(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
apoACP_c,EX_apoACP_c_,0.02651,0,0.00%
arg_L_e,EX_arg_L_e_,0.2409,6,0.56%
asn_L_e,EX_asn_L_e_,0.1815,4,0.28%
asp_L_e,EX_asp_L_e_,9.068,4,14.04%
ca2_e,EX_ca2_e_,0.003417,0,0.00%
cl_e,EX_cl_e_,0.003417,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.002278,0,0.00%
cu2_e,EX_cu2_e_,0.002278,0,0.00%
cys_L_e,EX_cys_L_e_,8.655,3,10.05%
fe3_e,EX_fe3_e_,0.01057,0,0.00%


In [66]:
model.reactions.get_by_id('UCYR')


Reaction identifier,UCYR
Name,R_UCYR
Memory address,0x01fcac8c2430
Stoichiometry,"2.0 focytC_p + q8h2_c <=> 2.0 ficytC_p + 2.0 h_p + q8_c 2.0 ferrocytochrome, cytc + Ubiquinol-8 <=> 2.0 ferricytochrome, cytc + 2.0 H + Ubiquinone-8"
GPR,NGO2031 and NGO2030 and NGO2029 and (NGO1080 or NGO0101 or Orphan)
Lower bound,-999999.0
Upper bound,999999.0


In [67]:
model.metabolites.get_by_id('q8_c')



Metabolite identifier,q8_c
Name,Ubiquinone-8
Memory address,0x01fcac31ad60
Formula,C49H74O4
Compartment,cytosol
In 12 reaction(s),"FUMR, NADH16pp, UCYR, L_LACD2, SUCD1, NADHQNa, NADPHQR2, NADH5, ASPO3, DHORD2, MDH2, LDH_D2"


In [68]:
rpmi_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_cobalt2_e_','EX_cu2_e_','EX_fe3_e_','EX_mn2_e_','EX_mobd_e_','EX_zn2_e_','EX_ca2_e_','EX_no3_e_','EX_h2o_e_','EX_k_e_','EX_cl_e_','EX_mg2_e_','EX_so4_e_','EX_na1_e_','EX_pi_e_','EX_h_e_','EX_arg_L_e_','EX_asn_L_e_','EX_asp_L_e_','EX_cys_L_e_','EX_glu_L_e_','EX_gly_e_','EX_his_L_e_','EX_ile_L_e_','EX_leu_L_e_','EX_lys_L_e_','EX_met_L_e_','EX_phe_L_e_','EX_pro_L_e_','EX_ser_L_e_','EX_thr_L_e_','EX_trp_L_e_','EX_tyr_L_e_','EX_val_L_e_','EX_pnto_R_e_','EX_chol_e_','EX_inost_e_','EX_glc_D_e_','EX_gthrd_e_','EX_co2_e_'])

def rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -0. 



mdm_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_co2_e_', 'EX_cobalt2_e_', 'EX_cu2_e_' , 'EX_h_e_', 'EX_h2o_e_','EX_mn2_e_','EX_mobd_e_','EX_tungs_e_', 'EX_zn2_e_','EX_na1_e_','EX_cl_e_','EX_k_e_','EX_so4_e_','EX_nh4_e_','EX_mg2_e_','EX_pi_e_','EX_ca2_e_','EX_fe3_e_','EX_no3_e_','EX_asp_L_e_','EX_glu_L_e_','EX_arg_L_e_','EX_gly_e_','EX_ser_L_e_','EX_leu_L_e_','EX_ile_L_e_','EX_val_L_e_','EX_tyr_L_e_','EX_cys_L_e_','EX_pro_L_e_','EX_trp_L_e_','EX_thr_L_e_','EX_phe_L_e_','EX_asn_L_e_','EX_gln_L_e_','EX_his_L_e_','EX_met_L_e_','EX_ala_L_e_','EX_lys_L_e_','EX_gthrd_e_','EX_thm_e_','EX_pnto_R_e_','EX_glc_D_e_','EX_hxan_e_','EX_ura_e_'])

def mdm(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -0.        

def mdm_with_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.              

def mdm_with_lactate_only(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10. 
            
            
def mdm_molarity_with_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -1000.
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -27.8
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.48
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -3.76
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.17
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -8.84
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.33
        if 'EX_ala_L_e_' in reaction.id:
            reaction.lower_bound = -1.12
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.51
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.69
        if 'EX_gln_L_e_' in reaction.id:
            reaction.lower_bound = -0.34
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.43
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -107.94697
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -11.48
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -5.74
        if 'EX_nh4_e_' in reaction.id:
            reaction.lower_bound = -4.11
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -1.07
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -40
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.254
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.051
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -0.71
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.23
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -0.65
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.42
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.13
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_thm_e_' in reaction.id:
            reaction.lower_bound = -0.00697
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.004
        if 'EX_hxan_e_' in reaction.id:
            reaction.lower_bound = -0.37
        if 'EX_ura_e_' in reaction.id:
            reaction.lower_bound = -0.45  
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -105.018
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -20
    

def mdm_molarity_with_lactate_only(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -1000.
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -0
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.48
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -3.76
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.17
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -8.84
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.33
        if 'EX_ala_L_e_' in reaction.id:
            reaction.lower_bound = -1.12
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.51
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.69
        if 'EX_gln_L_e_' in reaction.id:
            reaction.lower_bound = -0.34
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.43
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -107.94697
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -11.48
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -5.74
        if 'EX_nh4_e_' in reaction.id:
            reaction.lower_bound = -4.11
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -1.07
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -40
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.254
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.051
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -0.71
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.23
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -0.65
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.42
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.13
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_thm_e_' in reaction.id:
            reaction.lower_bound = -0.00697
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.004
        if 'EX_hxan_e_' in reaction.id:
            reaction.lower_bound = -0.37
        if 'EX_ura_e_' in reaction.id:
            reaction.lower_bound = -0.45  
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -105.018
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -20
               
def mdm_molarity(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -1000.
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -27.8
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.48
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -3.76
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.17
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -8.84
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.33
        if 'EX_ala_L_e_' in reaction.id:
            reaction.lower_bound = -1.12
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.51
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.69
        if 'EX_gln_L_e_' in reaction.id:
            reaction.lower_bound = -0.34
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.43
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -107.94697
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -11.48
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -5.74
        if 'EX_nh4_e_' in reaction.id:
            reaction.lower_bound = -4.11
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -1.07
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -40
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.254
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.051
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -0.71
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.23
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -0.65
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.42
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.13
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_thm_e_' in reaction.id:
            reaction.lower_bound = -0.00697
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.004
        if 'EX_hxan_e_' in reaction.id:
            reaction.lower_bound = -0.37
        if 'EX_ura_e_' in reaction.id:
            reaction.lower_bound = -0.45  
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -105.018
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -1000.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -0
                         
        
def complete(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=-1000.   
            

In [69]:
for reaction in model.reactions:
     if reaction.lower_bound < -4:
            reaction.lower_bound=-1000.
for reaction in model.reactions:
     if reaction.upper_bound > 4:
            reaction.upper_bound=1000.

In [70]:
mdm(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ala_L_e,EX_ala_L_e_,10,3,8.75%
apoACP_c,EX_apoACP_c_,0.1052,0,0.00%
arg_L_e,EX_arg_L_e_,0.9561,6,1.67%
asn_L_e,EX_asn_L_e_,0.7203,4,0.84%
ca2_e,EX_ca2_e_,0.01356,0,0.00%
cl_e,EX_cl_e_,0.01356,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.009041,0,0.00%
cu2_e,EX_cu2_e_,0.009041,0,0.00%
cys_L_e,EX_cys_L_e_,0.2806,3,0.25%
fe3_e,EX_fe3_e_,0.04196,0,0.00%


In [75]:
mdm_with_lactate_only(model)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ala_L_e,EX_ala_L_e_,10,3,9.75%
arg_L_e,EX_arg_L_e_,0.764,6,1.49%
asn_L_e,EX_asn_L_e_,0.5756,4,0.75%
ca2_e,EX_ca2_e_,0.01084,0,0.00%
cl_e,EX_cl_e_,0.01084,0,0.00%
cobalt2_e,EX_cobalt2_e_,0.007225,0,0.00%
cu2_e,EX_cu2_e_,0.007225,0,0.00%
cys_L_e,EX_cys_L_e_,0.18,3,0.18%
fe3_e,EX_fe3_e_,0.03354,0,0.00%
gln_L_e,EX_gln_L_e_,10,5,16.26%


In [79]:
#Check Gc model ability to grow anaerobically on nitrite. 
def anaerobic_rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_no2_e_':
            reaction.lower_bound = -10. 

anaerobic_rpmi(model)
print(model.slim_optimize())
print(model.metabolites.no_p.summary())

-2.095119304419103e-31
no_p
====
Formula: NO

Producing Reactions
-------------------
Percent  Flux Reaction                                             Definition
100.00% 1.709     NO2R ficytC_p + h2o_p + no_p <=> focytC_p + 2.0 h_p + no2_p

Consuming Reactions
-------------------
Percent   Flux Reaction                                                         Definition
100.00% -1.709      NOR 2.0 ficytC_p + 2.0 h_c + 2.0 no_p <=> 2.0 focytC_p + h2o_p + n2o_p


In [80]:
#NO2R for nitrite reductase (aniA) is switched in terms of ficytC and focytC, which prevents use of 
#nitrite to support anaerobic growth. This also allows spontaneous creation of energy through ficytC when grwon on NO.
#Correcting here. 

r=model.reactions.get_by_id('NO2R')
ficytC_p=model.metabolites.get_by_id('ficytC_p')
focytC_p=model.metabolites.get_by_id('focytC_p')

r.add_metabolites({
    ficytC_p: 2.0,
    focytC_p: -2.0
  
})
r


Reaction identifier,NO2R
Name,R_NO2R
Memory address,0x01fcac769790
Stoichiometry,"focytC_p + h2o_p + no_p <=> ficytC_p + 2.0 h_p + no2_p ferrocytochrome, cytc + H2O + Nitric-oxide <=> ferricytochrome, cytc + 2.0 H + Nitrite"
GPR,NGO1276 and NGO1328
Lower bound,-1000.0
Upper bound,1000.0


In [81]:
#Model now grows anaerobically on nitrite. 
anaerobic_rpmi(model)
print(model.slim_optimize())
model.metabolites.no_p.summary()

0.31597267853073796


Percent,Flux,Reaction,Definition
100.00%,10,NO2R,focytC_p + h2o_p + no_p <=> ficytC_p + 2.0 h_p + no2_p
Percent,Flux,Reaction,Definition
100.00%,-10,NOR,2.0 ficytC_p + 2.0 h_c + 2.0 no_p <=> 2.0 focytC_p + h2o_p + n2o_p


In [83]:
model.reactions.get_by_id('CYSabcpp')

Reaction identifier,CYSabcpp
Name,R_CYSabcpp
Memory address,0x01fcac53d6d0
Stoichiometry,atp_c + cys_L_p + h2o_c --> adp_c + cys_L_c + h_c + pi_c ATP + L-Cysteine + H2O --> ADP + L-Cysteine + H + Phosphate
GPR,(NGO0374 and NGO0373 and NGO0372) or (NGO0968 and NGO0269 and NGO1152)
Lower bound,0.0
Upper bound,1000.0


In [84]:
#NGO2011, NGO2012, NGO2013, and NGO2014 are a cysteine (cys_L_) transporter. PMID: 22138345
#NGO0372, NGO373, NGO0374 are a cystine (cystine_L_) transporter. PMID: 22138345. In the current model, this is annotated as a cysteine importer 
#and no reaction for cystine import exists, or for breakdown of cystine. 
#Adding that transport and breakdown reaction here and correcting the gene annotations.

#NGO0968 and NGO0269 and NGO1152 is listed for lots of different transport reactions - its not very well annotated. Removing it from cystine and cysteine import. 

CYSabcpp = model.reactions.get_by_id('CYSabcpp')
CYSabcpp.gene_reaction_rule='NGO2011 and NGO2012 and NGO2013 and NGO2014'
CYSabcpp.notes = {'citations': 'PMID: 22138345', 'type': '', 'annotation': ''}


from cobra import Reaction, Metabolite

cystine_L_c = Metabolite(
    'cystine_L_c',
    formula='C6H12N2O4S2',
    name='Cystine',
    compartment='cytosol')

cystine_L_e = Metabolite(
    'cystine_L_e',
    formula='C6H12N2O4S2',
    name='Cystine',
    compartment='extracellular')

cystine_L_p = Metabolite(
    'cystine_L_p',
    formula='C6H12N2O4S2',
    name='Cystine',
    compartment='periplasm')


h_c = model.metabolites.get_by_id('h_c')
atp_c = model.metabolites.get_by_id('atp_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
adp_c = model.metabolites.get_by_id('adp_c')
pi_c = model.metabolites.get_by_id('pi_c')

CYSTINEabcpp = cobra.Reaction('CYSTINEabcpp')
CYSTINEabcpp.id = 'CYSTINEabcpp'
CYSTINEabcpp.name = 'cystine import'
CYSTINEabcpp.gene_reaction_rule = 'NGO0374 and NGO0373 and NGO0372'
CYSTINEabcpp.lower_bound = -1000.
CYSTINEabcpp.upper_bound = 1000.
CYSTINEabcpp.add_metabolites({
        atp_c: -1.0,
        cystine_L_p: -1.,
        h2o_c: -1. ,
        adp_c:  1.,
        cystine_L_c: 1. ,
        h_c: 1.,
        pi_c: 1.,
        })

model.add_reactions([CYSTINEabcpp])






EX_cystine_L_ = cobra.Reaction('EX_cystine_L_')
EX_cystine_L_.notes = {'citations': 'PMID: 22138345', 'type': '', 'annotation': ''}
EX_cystine_L_.name = 'exchange of cystine'
EX_cystine_L_.gene_reaction_rule = ''
EX_cystine_L_.lower_bound = 0.
EX_cystine_L_.upper_bound = 1000.
EX_cystine_L_.add_metabolites({
    cystine_L_e: -1.0,})


model.add_reactions([EX_cystine_L_])


h_c = model.metabolites.get_by_id('h_c')
nadh_c = model.metabolites.get_by_id('nadh_c')
nad_c = model.metabolites.get_by_id('nad_c')
cys_L_c = model.metabolites.get_by_id('cys_L_c')

CYSR = cobra.Reaction('CYSR')
CYSR.notes = {'citations': 'PMCID: PMC2323225', 'type': '', 'annotation': ''}
CYSR.name = 'cystine reductase'
CYSR.gene_reaction_rule = 'Orphan'
CYSR.lower_bound = -1000.
CYSR.upper_bound = 1000.
CYSR.add_metabolites({
    cys_L_c: -2.0,
    nad_c: -1.0, 
    cystine_L_c: 1.0,
    h_c: 1.0,
    nadh_c: 1.0,
    })


model.add_reactions([CYSR])
print(CYSR.check_mass_balance())





{}


In [85]:
#Neisseria, like many other proteobacteria, lack the permease portion of the phosphotransferase. 
#And therefore do not use it for carbohydrate transport.
#The remaining portions of the system are thought to perform regulatory roles, related to virulence. 
#PMID: 26858137

model.remove_reactions(model.reactions.GLCptspp)
model.remove_reactions(model.reactions.FRUpts2pp)
model.remove_reactions(model.reactions.ASCBptspp)
model.remove_reactions(model.reactions.MALTptspp)
model.remove_reactions(model.reactions.MANptspp)
model.remove_reactions(model.reactions.SUCptspp)

C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [86]:
#NADPH does not catalyze the FabI reaction, only NADH. Therefore all of the NADPH dependent reactions were removed. 
#PMID: 26567338   
model.remove_reactions(model.reactions.EAR120y)
model.remove_reactions(model.reactions.EAR100y)
model.remove_reactions(model.reactions.EAR121y)
model.remove_reactions(model.reactions.EAR140y)
model.remove_reactions(model.reactions.EAR141y)
model.remove_reactions(model.reactions.EAR160y)
model.remove_reactions(model.reactions.EAR161y)
model.remove_reactions(model.reactions.EAR180y)
model.remove_reactions(model.reactions.EAR181y)
model.remove_reactions(model.reactions.EAR40y)
model.remove_reactions(model.reactions.EAR60y)
model.remove_reactions(model.reactions.EAR80y)

    

In [87]:
#Gc can only grow on glucose, lactate, and pyruvate. It cannot grow on gluconate. 
#Despite similarities to gluconate transporters and gluconate kinase, Nm also can't grow on gluconate so this should be removed. 
#PMID: 27454890
model.remove_reactions(model.reactions.GLCNtex)
model.remove_reactions(model.reactions.EX_glcn_e_)


In [88]:
#To remove an infeasible loop, where gln is exchanged infinitely between the two Na+ and h+ dependent transporters, limited these to import only.

model.reactions.GLNt2pp.lower_bound =0
model.reactions.GLNt1pp.lower_bound =0  

In [89]:
#D_ala import is predicted to occur through an orphan reaction and spontaneously
#Linking D_ala import to L_ala import genetically and na+ and h+ according to the transporters as appropriate


ALADt2pp =model.reactions.ALADt

ALADt2pp.id = 'ALADt2pp'
ALADt2pp.gene_reaction_rule = 'NGO1807 or NGO1787 or NGO1290 or NGO1957'

h_p = model.metabolites.get_by_id('h_p')
h_c = model.metabolites.get_by_id('h_c')
na1_c = model.metabolites.get_by_id('na1_c')
na1_p = model.metabolites.get_by_id('na1_p')
ala_D_p = model.metabolites.get_by_id('ala_D_p')
ala_D_c = model.metabolites.get_by_id('ala_D_c')

ALADt2pp.add_metabolites({
        na1_p: -1.0,
        na1_c: 1.0
        })

print(ALADt2pp.check_mass_balance())

ALADt1pp = cobra.Reaction('ALADt1pp')
ALADt1pp.name = 'D_ala import'
ALADt1pp.gene_reaction_rule = 'NGO1290 or NGO1787'
ALADt1pp.lower_bound = -1000.
ALADt1pp.upper_bound = 1000.
ALADt1pp.add_metabolites({
    ala_D_p: -1.0,
    h_p: -1.0,
    ala_D_c: 1.0,
    h_c: 1.0
    })


print(ALADt1pp.check_mass_balance())


{}
{}


In [90]:
#Remove looping reactions
model.reactions.ALAt1pp.lower_bound = 0
model.reactions.ALAt2pp.lower_bound = 0
model.reactions.ARGt1pp.lower_bound = 0
model.reactions.ARGt2pp.lower_bound = 0
model.reactions.ASNt1pp.lower_bound = 0
model.reactions.ASNt2pp.lower_bound = 0
model.reactions.ASPt1pp.lower_bound = 0
model.reactions.ASPt2pp.lower_bound = 0
model.reactions.CYSt1pp.lower_bound = 0
model.reactions.CYSt2pp.lower_bound = 0
model.reactions.GLUt1pp.lower_bound = 0
model.reactions.GLUt2pp.lower_bound = 0
model.reactions.HISt1pp.lower_bound = 0
model.reactions.HISt2pp.lower_bound = 0
model.reactions.ILEt1pp.lower_bound = 0
model.reactions.ILEt3pp.lower_bound = 0
model.reactions.LEUt1pp.lower_bound = 0
model.reactions.LEUt3pp.lower_bound = 0
model.reactions.LYSt1pp.lower_bound = 0
model.reactions.LYSt2pp.lower_bound = 0
model.reactions.METt1pp.lower_bound = 0
model.reactions.METt2pp.lower_bound = 0
model.reactions.PHEt1pp.lower_bound = 0
model.reactions.PHEt2pp.lower_bound = 0
model.reactions.PROt1pp.lower_bound = 0
model.reactions.PROt2rpp.lower_bound = 0
model.reactions.SERt1rpp.lower_bound = 0
model.reactions.SERt2pp.lower_bound = 0
model.reactions.THRt1rpp.lower_bound = 0
model.reactions.THRt2pp.lower_bound = 0
model.reactions.VALt1pp.lower_bound = 0
model.reactions.VALt3pp.lower_bound = 0
model.reactions.TRPS1.lower_bound=0
model.reactions.ADK1.lower_bound=0
model.reactions.NDPK1.lower_bound=0
model.reactions.VALTA.upper_bound=0
model.reactions.VPAMT.upper_bound=0

In [91]:
model

Name,NGO_513
Memory address,0x01fca4858e20
Number of metabolites,1341
Number of reactions,1433
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"periplasm, cytosol, extracellular, C_c, C_p, c"


In [92]:
for metabolite in model.metabolites:
    if '_c' in metabolite.id:
        metabolite.compartment = 'cytosol'
    if '_p' in metabolite.id:
        metabolite.compartment = 'periplasm'
    if '_e' in metabolite.id:
        metabolite.compartment = 'extracellular'

In [93]:
model

Name,NGO_513
Memory address,0x01fca4858e20
Number of metabolites,1341
Number of reactions,1433
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"periplasm, cytosol, extracellular"


In [98]:
len(model.genes)
#Subtract 2 for "Sgene, and orphan" 

559

In [99]:
model.id='iNGO_557'

In [103]:
cobra.io.write_sbml_model(model, cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/NGO_557.sbml')

In [104]:
cobra.io.save_json_model(model, cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/NGO_557.json')